In [1]:
import pandas as pd
import numpy as np
import torch

from eugene.dataload._io import read_numpy, read

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic import
import eugene as eu
eu.__version__

Global seed set to 13


GPU is available: True
Number of GPUs: 1
Current GPU: 0
GPUs: NVIDIA GeForce RTX 2070


'0.0.6'

In [2]:
sdata = read_numpy("chr1.npy_ohe_seqs.npy", ohe=True)

In [3]:
eu.pp.train_test_split_sdata(sdata)

SeqData object modified:
    seqs_annot:
        + train_val


In [4]:
seq_len = 50
latent_dim = seq_len * 4

In [5]:
generator = torch.nn.Sequential(
    eu.models.base.BasicFullyConnectedModule(
        input_dim=500,
        output_dim=latent_dim,
        hidden_dims=[300]
    ),
    torch.nn.Tanh()
)

In [6]:
disc_batchnorm = True
disc_conv_dropout = 0
disc_fc_dropout = 0
discriminator = torch.nn.Sequential(
    eu.models.DeepBind(
        input_len=latent_dim,
        output_dim=1,
        pool_width=8,
        conv_kwargs=dict(channels=[4, 128], conv_kernels=[16], dropout_rates=disc_conv_dropout, batchnorm=disc_batchnorm),
        fc_kwargs=dict(hidden_dims=[256, 64], dropout_rate=disc_fc_dropout, batchnorm=disc_batchnorm),
        mode="dna",
    ),
    torch.nn.Sigmoid()
)

In [7]:
model = eu.models.GAN(
    latent_dim=latent_dim, 
    generator=generator, 
    discriminator=discriminator,
    grad_clip=0.01,
    gen_lr=1e-3,
    disc_lr=1e-3,
)

In [8]:
eu.settings.batch_size = 256
eu.settings.dl_num_workers = 8
eu.settings.dl_pin_memory_gpu_training = True
eu.train.fit(
    model=model,
    sdata=sdata,
    early_stopping_callback=False,
)

Global seed set to 13
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


No transforms given, assuming just need to tensorize.
No transforms given, assuming just need to tensorize.



  | Name          | Type       | Params
---------------------------------------------
0 | generator     | Sequential | 210 K 
1 | discriminator | Sequential | 157 K 
---------------------------------------------
367 K     Trainable params
0         Non-trainable params
367 K     Total params
1.470     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 13


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

c:\Users\Lab\anaconda3\envs\eugene\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


---